Imports and other things to start the project

In [6]:
!pip install fastparquet

   ---------------------------------------- 0.0/671.0 kB ? eta -:--:--
   --------------------------------------- 671.0/671.0 kB 13.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 14.6 MB/s eta 0:00:00


In [7]:
import pandas as pd
import re
import psutil
import json
import urllib.request
import fastparquet

In order to properly use this data, we must start by importing it and then preprocessing

In [8]:
file_path = "train-00000-of-00206.parquet"

df = pd.read_parquet(file_path, engine = "fastparquet")

first_entry_code = df.iloc[0]["content"]
second_entry_code = df.iloc[100]["content"]
third_entry_code = df.iloc[1000]["content"]

print(df)
print(first_entry_code)
print(second_entry_code)
print(third_entry_code)

                                          hexsha   size ext    lang  \
0       f70001f658d4dfaa72dd4f0d1b3176492f6658bb   6442  py  Python   
1       f7000273e22d5a0f2d5b40c38a0ed8511d1b8995   2250  py  Python   
2       f70002926d1d600b4b068459c9dd40ebf3aef47d    757  py  Python   
3       f7000327daf9ff11a381ce6d5de401ff007d1323   1094  py  Python   
4       f7000371f0315cd55c0b14b33e7e8e56697cfc2e  10498  py  Python   
...                                          ...    ...  ..     ...   
117540  793e7c8154dde86884b9669622b32aeea1185d21   1853  py  Python   
117541  793e7d1ab6c0471710bdbffc3c1b2a68e0aa3516    948  py  Python   
117542  793e7e0a68202e76b818772ff6d01655ca87546e   2485  py  Python   
117543  793e7ea61a6d183a0434da1b893d3870cec79aba   1406  py  Python   
117544  793e7fc5c6f5a39a1fb0909ab5a3e5ce33f87ab8  25955  py  Python   

                                      max_stars_repo_path  \
0                                       spider/openwrt.py   
1                        

In [9]:
# Get only the code

code_series = df["content"].head(1000)
print(code_series[1])

# UCF Senior Design 2017-18
# Group 38

from PIL import Image
import cv2
import imagehash
import math
import numpy as np

DIFF_THRES = 20
LIMIT = 2
RESIZE = 1000


def calc_hash(img):
    """
    Calculate the wavelet hash of the image
        img: (ndarray) image file
    """
    # resize image if height > 1000
    img = resize(img)
    return imagehash.whash(Image.fromarray(img))


def compare(hash1, hash2):
    """
    Calculate the difference between two images
        hash1: (array) first wavelet hash
        hash2: (array) second wavelet hash
    """
    return hash1 - hash2


def limit(img, std_hash, count):
    """
    Determine whether image should be removed from image dictionary in main.py
        img: (ndarray) image file
        std_hash: (array) wavelet hash of comparison standard
        count: (int) global count of images similar to comparison standard
    """
    # calculate hash for given image
    cmp_hash = calc_hash(img)

    # compare to standard
    diff = compar

In [ ]:
# Preprocess

def clean_code(code):
    # Remove multiline docstrings 
    code = re.sub(r'"""[\s\S]*?"""', '', code)
    code = re.sub(r"'''[\s\S]*?'''", '', code)

    # Remove single-line comments 
    code = re.sub(r'#.*', '', code)

    # Remove blank lines
    code = re.sub(r'^\s*\n', '', code)

    return code

In [ ]:
cleaned_data = code_series.apply(clean_code)

print(cleaned_data[1])

from PIL import Image
import cv2
import imagehash
import math
import numpy as np

DIFF_THRES = 20
LIMIT = 2
RESIZE = 1000


def calc_hash(img):
    
    
    img = resize(img)
    return imagehash.whash(Image.fromarray(img))


def compare(hash1, hash2):
    
    return hash1 - hash2


def limit(img, std_hash, count):
    
    
    cmp_hash = calc_hash(img)

    
    diff = compare(std_hash, cmp_hash)

    
    if diff <= DIFF_THRES:
        
        if count >= LIMIT:
            return 'remove'

    
    else:
        
        return 'update_std'

    
    return 'continue'


def resize(img):
    
    
    width = np.shape(img)[1]
    height = np.shape(img)[0]

    
    if width > RESIZE:
        
        scale = RESIZE / width
        resized_img = cv2.resize(
            img, (RESIZE, math.floor(height / scale)), cv2.INTER_AREA)
        
        return resized_img

    
    return img


def set_standard(images, filename):
    
    return filename, calc_hash(images[filename]), 0



In [ ]:
#Save as json
cleaned_data_list = cleaned_data.tolist()

formatted_data = [{"code": item} for item in cleaned_data_list]

payload = json.dumps(formatted_data, ensure_ascii=False).encode("utf-8")

output_filename = "cleaned_data.json"

with open(output_filename, "w") as f:
    json.dump(formatted_data, f, indent=4)

print(f"Data saved to {output_filename}")


Data saved to cleaned_data.json


Generate summaries for data

In [13]:
#Check ollama
import psutil

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [ ]:
import urllib.request

def query_model(
    prompt,
    model="qwen2.5-coder:7b",
    url="http://localhost:11434/api/chat"
):
    
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "options": {     
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }


    payload = json.dumps(data).encode("utf-8")

    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )
    request.add_header("Content-Type", "application/json")

   
    response_data = ""
    with urllib.request.urlopen(request) as response:
        
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data


model = "qwen2.5-coder:7b"
result = query_model("What do Llamas eat?", model)
print(result)

Llamas primarily eat grasses and hay. They are herbivores and have a digestive system that allows them to digest cellulose, which is found in plant material. In the wild, llamas graze on grasses, leaves, and bark from trees and bushes. In captivity or when kept as pets, they are usually fed commercial llama pellets, supplemented with fresh grass and hay.


In [ ]:
def add_summaries_to_json(input_json_filename, output_json_filename):
    with open(input_json_filename, "r") as infile:
        code_data = json.load(infile)
    
    if not isinstance(code_data, list):
        raise ValueError("Expected a list of dictionaries but got a different format.")
    
    
    id = 0
    for entry in code_data:
        if not isinstance(entry, dict) or "code" not in entry:
            raise ValueError(f"Invalid entry format: {entry}")
        
        prompt = f"Summarize the following Python code:\n{entry['code']}"
        summary = query_model(prompt)
        entry["summary"] = summary
        print(f"ID: {id}")
        print(f"Entry: {entry}")
        print("Summary: " + summary)
        print()
        print()
        id = id + 1

   
    with open(output_json_filename, "w") as outfile:
        json.dump(code_data, outfile, indent=4)


In [ ]:
# Proof of concept

testdf = df["content"].head(10)

cleaned_test = testdf.apply(clean_code)

cleaned_data_list_test = cleaned_test.tolist()

formatted_data_test = [{"code": item} for item in cleaned_data_list_test]

payload_test = json.dumps(formatted_data_test, ensure_ascii=False).encode("utf-8")

output_filename = "cleaned_data_test.json"

with open(output_filename, "w") as f:
    json.dump(formatted_data_test, f, indent=4)

print(f"Data saved to {output_filename}")

add_summaries_to_json("cleaned_data_test.json", "summarized_code_test.json")

print("Summaries added and saved to summarized_code_test.json")

Data saved to cleaned_data_test.json
ID: 0
Entry: {'code': 'from   _TFL.pyk           import pyk\n\nfrom   rsclib.HTML_Parse  import tag, Page_Tree\nfrom   rsclib.autosuper   import autosuper\nfrom   spider.common      import Interface, Inet4, Inet6, unroutable\nfrom   spider.common      import WLAN_Config\nfrom   spider.luci        import Version_Mixin\n\nclass Status (Page_Tree, Version_Mixin) :\n    url          = \'cgi-bin/luci/freifunk/status/status\'\n    retries      = 2\n    timeout      = 10\n    html_charset = \'utf-8\' \n\n    wl_names = dict \\\n        ( ssid    = \'ssid\'\n        , _bsiid  = \'bssid\'\n        , channel = \'channel\'\n        , mode    = \'mode\'\n        )\n\n    def parse (self) :\n        root  = self.tree.getroot ()\n        self.wlans  = []\n        self.routes = {}\n        for div in root.findall (".//%s" % tag ("div")) :\n            id = div.get (\'id\')\n            if id == \'cbi-wireless\' :\n                wlan_div = div\n            elif i

In [ ]:
add_summaries_to_json("cleaned_data.json", "summarized_code.json")

print("Summaries added and saved to summarized_code.json")